# **Extracción de recuerdos de videos**

## Sistema de extracción y almacenamiento de grafos a partir de una entrevista.

## https://github.com/Hugarc02/TFG2022ExtraccionRecuerdos







# **Instrucciones**

Para usar el sistema, ejecute las celdas en orden y rellene los formularios. A continuación se explica cada formulario.
<br>
<br>
### **Formulario directorio de trabajo** 

En este formulario se establece el directorio de trabajo para la ejecución.

- `path_carpeta_ejecucion`: ruta a la carpeta de google drive donde se almacena el código y archivos necesarios.
<br>
<br>

### **Formulario opciones de transcripción** 

En este formulario se elige el modo de transcripción.

- `modo_transcripcion`: Seleccionar Automática o Manual para elegir el tipo de transcripción.
<br>
<br>

### **Formulario transcripción Automática:**

Rellene este formulario si ha seleccionado la transcripción automática.

Primero hay que preparar la transcripción asíncrona de Google Cloud, siga los siguientes pasos.
- Registrarse en Google Cloud
- Crear proyecto en Google Cloud
- Habilitar la API Google Speech to Text
- Crear cuenta de servicio con rol "Administrador de almacenamiento"
- Crear y descargar **clave JSON** de la cuenta de servicio
- Crear **bucket de almacenamiento** en Google Cloud Storage
<br>

Habiendo realizado estos pasos, rellene los campos el formulario. 

- `formato_video`: Seleccionar Video o Audio según el formato del archivo a transcribir.
- `path_archivo`: Ruta a el archivo a transcribir.
- `path_clave`: Ruta a la **clave JSON** de la cuenta de servicio.
- `nombre_bucket`: Nombre del **bucket de almacenamiento**.

Formatos de video aceptados: mp4, mkv, ogv, webm, avi, mov.

Formatos de audio aceptados: ogg, mp3, wav, p4a.
<br>
<br>

### **Formulario transcripción manual:**

Rellene este formulario si ha seleccionado la transcripción manual.

- `texto`: Texto de la transcripción.
<br>
<br>

### **Formulario opciones extracción de grafos:**

En este formulario se eligen las opciones de la extracción de grafos.
- `guion`: Ruta a el archivo JSON del guión.
- `mostrar_segmentos`: Marcar para que el sistema muestre los segmentos de la entrevista procesados.
- `mostrar_consultas`: Marcar para que el sistema muestre las consultas obtenidas.
- `subir_neo4j`: Marcar para que el sistema suba las consultas a Neo4j.
<br>
<br>

### **Formulario Neo4j:**

Rellene este formulario si ha marcado el campo `subir_neo4j`.

En este formulario se añaden los datos necesarios para establecer una conexión con una base de datos Neo4j.
- `usuario`: Nombre de usuario de base de datos Neo4j.
- `contraseña`: Contraseña de usuario de base de datos Neo4j.
- `uri`: Uri de base de datos Neo4j.






In [ ]:
#@title # Instalaciones
#@markdown Nota: es necesario reiniciar el runtime tras instalar para poder usar las instalaciones. El avisó saldrá debajo de la celda con un botón "Restart Runtime".

!pip install spacy==3.2.3
!pip install pyyaml==6.0
!python -m spacy download es_core_news_lg
!python -m spacy download es_dep_news_trf
!pip install networkx==2.6.3
!pip install neo4j==4.4.2
!pip install --upgrade google-cloud-speech
!pip install pydub==0.25.1
!pip install moviepy==1.0.3

!python -m nltk.downloader wordnet_ic
!python -m nltk.downloader omw-1.4
!python -m nltk.downloader wordnet

!pip install ipywidgets
!pip install protobuf==3.20.*

In [ ]:
#@title # Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title # Directorio de trabajo

import os
path_carpeta_ejecucion = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab' #@param {type:"string"}
#@markdown Consejo: En el explorador de archivos de Google Colab puedes obtener el path a una carpeta o archivo haciendo click derecho y seleccionando "copiar ruta"
path_carpeta_ejecucion.encode()
os.chdir(path_carpeta_ejecucion)

In [ ]:
#@title # Módulo de transcripción

import os
import moviepy.editor as mp
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech
from pydub import AudioSegment, effects, scipy_effects

def uploadAudio(file, fileFormat, credentialsPath, bucketName):
    # Load credentials in enviromental variable
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentialsPath
    
    # Connect to the bucket
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucketName)
    blob = bucket.blob('tempAudio.wav')
    
    # Transform video to audio
    if fileFormat:
        # If file is video
        clip = mp.VideoFileClip(file)
        clip.audio.write_audiofile(r"tempAudio.wav")
    else:
        # If file is audio
        clip = mp.AudioFileClip(file)
        clip.write_audiofile(r"tempAudio.wav")
    
    # Normalize audio
    audio = AudioSegment.from_wav(r"tempAudio.wav")
    audio = scipy_effects.band_pass_filter(audio, 200, 3100)
    audio = audio.set_channels(1)
    audio = effects.normalize(audio)
    audio.export(r"tempAudio.wav", format = 'wav')
    # Upload audio to bucket
    blob.upload_from_filename('tempAudio.wav')
    
    # Delete audio file
    os.remove(r"tempAudio.wav")
    
def transcribeAudio(bucketName):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    gcs_uri = 'gs://' + bucketName + '/tempAudio.wav'
    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig( 
        language_code = 'es-ES',
        enable_speaker_diarization=True,
        diarization_speaker_count=2,
        enable_automatic_punctuation=True,
        )
    
    response = client.long_running_recognize(config=config, audio=audio)

    result = response.result().results[-1]

    words_info = result.alternatives[0].words

    # Printing out the output:
    transcription = ""
    curr_s = 0
    curr_t = ''
    for word_info in words_info:
        word_info.word, word_info.speaker_tag
        if curr_s != word_info.speaker_tag:
            transcription = transcription + curr_t +"\n"
            curr_s = word_info.speaker_tag
            curr_t = word_info.word
        else:
            curr_t = curr_t + ' ' + word_info.word
    transcription = transcription + curr_t +"\n"
    
    # Connect to the bucket
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucketName)
    blob = bucket.blob('tempAudio.wav')
    blob.delete()
    return transcription

In [ ]:
#@title # Módulo auxiliar de extracción de grafos

#SPACY
import spacy
from spacy.matcher import Matcher

#create  nlp object
nlp = spacy.load("es_core_news_lg")

#GRAFENO
from grafeno import pipeline
from grafeno.transformers import get_pipeline
from grafeno.jupyter import visualize
import yaml

#NEO4J
from neo4j import GraphDatabase

import os

#checks if two texts are similar
def similar(txt1, txt2):
    doc1 = nlp(txt1.lower().replace('¿', '').replace('?', ''))
    doc2 = nlp(txt2.lower().replace('¿', '').replace('?', ''))
    #print(txt1)
    #print(txt2)
    #print('SIMILITUD: ',doc1.similarity(doc2))
    if doc1.similarity(doc2) > 0.8:
        return doc1.similarity(doc2)
    else:
        sep = txt2.split(", ")
        for s in sep:
            s = doc1.similarity(nlp(s))
            if s > 0.8:
                return s
        return 0

def createGraphQuery(text, questionNumber):
    # Load Grafeno pipeline config
    config = yaml.safe_load(open(os.getcwd() + '/configs/custom.yaml'))
    
    # Run the pipeline on the text to obtain Neo4j query representin the graph obtained
    query = pipeline.run({ **config, 'text': text })
    
    return(query.replace('PX', 'P' + str(questionNumber)))

def createDateQuery(text, questionNumber):
    lines = text.splitlines()
    answer = ''
    if len(lines) > 1:
        for i in range(1, len(lines)):
            answer = answer + '. ' + lines[i]
    # Create doc
    doc = nlp(answer)

    # Initialize the matcher with the shared vocab
    matcher = Matcher(nlp.vocab)

    # Add date recognition patterns to the matcher
    # Year pattern (from 1900 to 2199)
    YearPattern = [[{"TEXT": {"REGEX": "(19|20|21)\d\d"}}]]
    # Day of the month pattern (ex: 23 de enero)
    DayMonthPattern = [[{"IS_DIGIT": True}, {"LOWER": "de"}, {"LOWER": {"REGEX": "^(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)$"}}]]
    # Month pattern (from enero to diciembre)
    MonthPattern = [[{"LOWER": {"REGEX": "^(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)$"}}]]
    # Day of the week pattern (from lunes to martes)
    DayPattern = [[{"LOWER": {"REGEX": "^(lunes|martes|miercoles|jueves|viernes|sabado|domingo)$"}}]]
    matcher.add("year", YearPattern)
    matcher.add("day", DayMonthPattern)
    matcher.add("month", MonthPattern)
    matcher.add("dayWeek", DayPattern)
    
    # Call the matcher on the doc
    matches = matcher(doc)

    # Iterate over the matches to find dates
    dates = {}
    for match_id, start, end in matches:
        # Get the matched span
        matched_span = doc[start:end]
        dates[doc.vocab.strings[match_id]] = matched_span.text
        
    # Insert matches into Neo4j cypher query
    # It merges a node with the matches as properties
    # Creates a relationship with the verb node of the same question, if it exists
    query = ''
    if dates and len(dates)>0:
        query = ('MERGE (n:DATE {' +', '.join('{}: {!r}'.format(k, str(dates[k])) for k in dates) +
                '}) SET n:PX, n._temp_id=\'0\';\n' + 
                'MATCH (n:PX:VERB), (m {_temp_id: \'0\'}) CREATE (n)-[r:CONTEXT]->(m);\n' +
                'MATCH (n) REMOVE n._temp_id;')
    return createGraphQuery(lines[0], questionNumber) + '\n' + query.replace('PX', 'P' + str(questionNumber))
    
def createTextQuery(text, questionNumber):
    lines = text.splitlines()
    answer = ''
    if len(lines) > 1:
        for i in range(1, len(lines)):
            answer = answer + '. ' + lines[i]
    query = 'CREATE (n:TEXT:PX {{concept: \'{}\'}});'.format(answer);
    return query.replace('PX', 'P' + str(questionNumber))

#produces the results based on the mode
def process(text, questionNumber, mode):
    if mode == 0: return createGraphQuery(text, questionNumber)
    elif mode == 1: return createDateQuery(text, questionNumber)
    elif mode == 2: return createTextQuery(text, questionNumber)

    


In [ ]:
#@title # Módulo de extracción de grafos

import json

def extractGraphs(lines, scriptPath):
    #load questions and results modes on dict
    with open(scriptPath, encoding='utf-8') as f:
        script = json.load(f)['preguntas']

    #get questions
    questions = list(script.keys())
    results = {}

    # Find the positions of the questions in the text lines, using similarity
    matches = [-1] * len(lines)
    for j in range(len(questions)):
        maxS = 0
        maxI = -1
        for i in range(len(lines)):
            if matches[i] == -1:
                s = similar(questions[j], lines[i])
                if s > maxS and s > 0.7:
                    maxS = s
                    maxI = i
        if maxI != -1:
            matches[maxI] = j

    segment = ''
    queries = []
    segments = []

    # Process each question to obtain queries
    qNum = -1
    for i in range(len(lines)):
        if qNum != -1:
            if matches[i] != -1:
                segments.append(segment)
                queries.append(process(segment, qNum, script[questions[qNum]]))
                #print(segment + '\n\n')
                qNum = matches[i]
                segment = lines[i]
            else:
                segment = segment + '\n' + lines[i]

        elif matches[i] != -1:
            qNum = matches[i]
            segment = lines[i].replace('¿', '').replace('?', '.')
        i += 1
    segments.append(segment)
    queries.append(process(segment, j, script[questions[j]]))
    #print(segment + '\n\n')    
    return (queries, segments)

def submitQueries(queries, user, pwd, uri):
    data_base_connection = GraphDatabase.driver(uri = uri, auth=(user, pwd))
    session = data_base_connection.session()   
    for qs in queries:
        for q in qs.splitlines():
            if q != '':
                session.run(q)
    session.run('match ()-[r]->() match (s)-[r]->(e) with s,e,type(r) as typ, tail(collect(r)) as coll foreach(x in coll | delete x);')


In [ ]:
#@title # Opciones de transcripción:
from ipywidgets import widgets

modo_transcripcion = 'Manual' #@param ["Automatica", "Manual"]

#@markdown ---
#@markdown ### Modo manual:

#@markdown Al ejecutar transcripción aparecerá un area de texto para insertarla.

#@markdown ---
#@markdown ### Modo automático:

formato_archivo = 'Audio' #@param ["Video", "Audio"]

path_archivo = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/audio_ejemplo.mp3' #@param {type:"string"}

path_clave = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/tfg-2021-2022-storage.json' #@param {type:"string"}

nombre_bucket = 'bucket-tfg-2021-2022' #@param {type:"string"}

#@markdown Consejo: En el explorador de archivos de Google Colab puedes obtener el path a una carpeta o archivo haciendo click derecho y seleccionando "copiar ruta"

In [ ]:
#@title # Ejecutar Transcripción

#@markdown Nota: para que el sistema funcione bien, las preguntas y respuestas deben estar separadas en lineas distintas. Si la separacion de hablantes automática no funciona, conviene que separar las líneas manualmente en el area de texto.


if modo_transcripcion == 'Manual':
    transcription = ""
elif modo_transcripcion == 'Automatica':
    uploadAudio(path_archivo, formato_archivo != 'Audio',  path_clave, nombre_bucket)
    transcription = transcribeAudio(nombre_bucket)
area = widgets.Textarea(
    value = transcription,
    layout=widgets.Layout(height="300px", width="50%")
    )
out = widgets.Output()
def select(change):
    out.clear_output()
    with out:
      for l in change.new.splitlines():
        display(l)
  
area.observe(select, names="value")
display(area)
print('\033[1m' + 'Transcripción: \n' + '\033[0m')
display(out)


In [ ]:
#@title # Opciones de extracción de grafos:


guion = '/content/drive/Shareddrives/TFG 21-22 extraccio\u0301n de recuerdos de videos/Versio\u0301n final colab/guion_entrevista.json' #@param {type:"string"}

mostrar_segmentos = True #@param {type:"boolean"}
mostrar_consultas = True #@param {type:"boolean"}
subir_neo4j = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### Neo4j:

usuario = 'neo4j' #@param {type:"string"}
contraseña = 'password' #@param {type:"string"}
uri = 'bolt://XX.XXX.XXX.XXX:XXXX' #@param {type:"string"}

In [ ]:
#@title # Ejecutar extracción de grafos

results = extractGraphs(area.value.splitlines(), guion)
if mostrar_segmentos or mostrar_consultas:
    for i in range(len(results[0])):
            print('\033[1m' + 'Pregunta ' + str(i+1) + '\033[0m' + '\n')
            if mostrar_segmentos: print('\033[94m' + 'Segmento:  \n' + results[1][i] + '\033[0m'+ '\n')
            if mostrar_consultas: print('\033[36m' + 'Consulta:  \n' + results[0][i] + '\033[0m' + '\n')
if subir_neo4j:
    submitQueries(results[0], usuario, contraseña, uri)